In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.optimize import minimize
from pypfopt import expected_returns, risk_models, EfficientFrontier

In [2]:
tickers = ('AAPL', 'NVDA')

In [5]:
conn = sqlite3.connect('sp500_stocks.db')
cursor = conn.cursor()
cursor.execute(f"SELECT ticker, date, close FROM price WHERE ticker IN {tuple(tickers)} AND date BETWEEN '2025-01-01' AND '2025-08-01'")
data = cursor.fetchall()
conn.close()
df = pd.DataFrame(data, columns=['ticker', 'date', 'close'])
df = df.pivot(index='date', columns='ticker', values='close').dropna()
df

ticker,AAPL,NVDA
date,,
2025-01-02 00:00:00,243.263199,138.287674
2025-01-03 00:00:00,242.774368,144.446686
2025-01-06 00:00:00,244.410416,149.405869
2025-01-07 00:00:00,241.627136,140.117371
2025-01-08 00:00:00,242.115936,140.087387
...,...,...
2025-07-25 00:00:00,213.880005,173.500000
2025-07-28 00:00:00,214.050003,176.750000
2025-07-29 00:00:00,211.270004,175.509995


In [9]:
df['Value']=0

In [10]:
allocation = [50,50]

In [14]:
for i in range(len(tickers)):
    df['Value'] += (1 + df.iloc[:,i].pct_change()).cumprod() * (1000 * allocation[i] /100)

In [15]:
df

ticker,AAPL,NVDA,Value
date,,,
2025-01-02 00:00:00,243.263199,138.287674,NaN
2025-01-03 00:00:00,242.774368,144.446686,1021.264103
2025-01-06 00:00:00,244.410416,149.405869,1042.557488
2025-01-07 00:00:00,241.627136,140.117371,1003.252803
2025-01-08 00:00:00,242.115936,140.087387,1004.149067
...,...,...,...
2025-07-25 00:00:00,213.880005,173.500000,1066.921657
2025-07-28 00:00:00,214.050003,176.750000,1079.021936
2025-07-29 00:00:00,211.270004,175.509995,1068.824535


In [310]:
returns = df.pct_change().dropna()
returns

ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,XYZ,YUM,ZBH,ZBRA,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2024-03-28 00:00:00,-0.012621,-0.010559,0.009703,-0.008713,0.001586,0.010494,0.016630,0.000397,0.023069,-0.002858,...,-0.000557,0.004323,0.006931,0.011046,-0.001082,0.016831,0.009685,-0.004751,0.007621,0.004154
2024-04-01 00:00:00,0.001969,-0.008456,-0.007359,-0.009639,-0.013813,-0.001406,-0.021465,-0.004974,-0.005915,-0.007324,...,-0.009746,0.042355,-0.008744,0.006452,-0.004643,-0.036888,0.013127,-0.016896,-0.001460,-0.012942
2024-04-02 00:00:00,-0.009550,-0.006999,-0.000387,-0.019955,-0.000624,-0.008883,-0.007990,-0.005736,-0.008850,0.005613,...,-0.003656,-0.012200,0.015015,0.019574,-0.004042,-0.027621,0.001708,-0.009865,-0.014950,-0.012034
2024-04-03 00:00:00,-0.003052,0.004797,-0.018595,-0.004809,-0.006606,0.008088,-0.012423,-0.004407,0.002463,-0.000797,...,-0.013830,0.011020,0.000185,0.000168,0.005776,0.005555,-0.012082,0.011987,-0.012648,-0.012363
2024-04-04 00:00:00,-0.016628,-0.004892,-0.053178,-0.003138,-0.010514,-0.012361,-0.005447,-0.019919,-0.017455,0.009417,...,-0.008586,-0.002819,-0.005916,0.003520,-0.012494,-0.061770,-0.019639,-0.018845,0.000717,0.015647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-29 00:00:00,0.002510,-0.012988,0.014322,-0.037423,0.013138,-0.004393,0.006714,0.003329,-0.000087,-0.010226,...,0.002318,-0.016986,0.016868,0.012922,-0.002268,-0.029664,0.006168,-0.010731,-0.009232,0.001725
2025-07-30 00:00:00,-0.014937,-0.010508,-0.009989,-0.015273,0.004218,-0.005922,-0.017569,-0.018074,0.001560,0.018450,...,-0.015420,0.001371,0.000691,-0.008682,-0.010306,-0.007547,0.012467,-0.017167,0.002382,-0.014569
2025-07-31 00:00:00,-0.027446,-0.007080,-0.001532,-0.017365,-0.018359,0.005256,-0.025182,-0.017335,-0.028039,-0.018478,...,-0.019186,-0.004565,0.014505,-0.002324,0.107351,-0.004253,-0.019321,-0.017895,0.019946,-0.020294


In [9]:
df['Change'] = df['AAPL'].pct_change()
(df['Change'] + 1).cumprod()

date
2024-03-27 00:00:00         NaN
2024-03-28 00:00:00    0.989441
2024-04-01 00:00:00    0.981074
2024-04-02 00:00:00    0.974208
2024-04-03 00:00:00    0.978882
                         ...   
2025-07-29 00:00:00    1.226392
2025-07-30 00:00:00    1.213505
2025-07-31 00:00:00    1.204914
2025-08-01 00:00:00    1.174787
2025-08-04 00:00:00    1.180417
Name: Change, Length: 339, dtype: float64

In [311]:
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

c:\Users\nolan\AppData\Local\Programs\Python\Python310\lib\site-packages\pypfopt\risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(
c:\Users\nolan\AppData\Local\Programs\Python\Python310\lib\site-packages\pypfopt\risk_models.py:89: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn(


In [312]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe(risk_free_rate=0.05)
cleaned_weights = ef.clean_weights()
print("Optimal Weights:", cleaned_weights)

Optimal Weights: OrderedDict([('A', 0.0), ('AAPL', 0.0), ('ABBV', 0.0), ('ABNB', 0.0), ('ABT', 0.0), ('ACGL', 0.0), ('ACN', 0.0), ('ADBE', 0.0), ('ADI', 0.0), ('ADM', 0.0), ('ADP', 0.0), ('ADSK', 0.0), ('AEE', 0.0), ('AEP', 0.0), ('AES', 0.0), ('AFL', 0.0), ('AIG', 0.0), ('AIZ', 0.0), ('AJG', 0.0), ('AKAM', 0.0), ('ALB', 0.0), ('ALGN', 0.0), ('ALL', 0.0), ('ALLE', 0.0), ('AMAT', 0.0), ('AMCR', 0.0), ('AMD', 0.0), ('AME', 0.0), ('AMGN', 0.0), ('AMP', 0.0), ('AMT', 0.0), ('AMZN', 0.0), ('ANET', 0.0), ('AON', 0.0), ('AOS', 0.0), ('APA', 0.0), ('APD', 0.0), ('APH', 0.0), ('APO', 0.0), ('APTV', 0.0), ('ARE', 0.0), ('ATO', 0.0), ('AVB', 0.0), ('AVGO', 0.0), ('AVY', 0.0), ('AWK', 0.0), ('AXON', 0.0), ('AXP', 0.0), ('AZO', 0.0), ('BA', 0.0), ('BAC', 0.0), ('BALL', 0.0), ('BAX', 0.0), ('BBY', 0.0), ('BDX', 0.0), ('BEN', 0.0), ('BF-B', 0.0), ('BG', 0.0), ('BIIB', 0.0), ('BK', 0.0), ('BKNG', 0.0), ('BKR', 0.0), ('BLDR', 0.0), ('BLK', 0.0), ('BMY', 0.0), ('BR', 0.0), ('BRK-B', 0.0), ('BRO', 0.0), 

In [313]:
ef.portfolio_performance(verbose=True, risk_free_rate=0.05)

Expected annual return: 99.4%
Annual volatility: 15.3%
Sharpe Ratio: 6.19


(0.994445671354342, 0.1526726367766476, 6.18608344818213)